In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import scipy as sp
import keras as ks
import tensorflow as tf
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler

# Now let fit a Machine Learning model(s)

What is the good apprach to fitting a model to our dataset? What appraches should we use? Logistic Regresson, Classification tree, SVM, Random Forrest? what about imbalanced data, how should we deal with it? ignore it, oversample, undersample how do we do cross-valdation if we over/under smaple?

In [ ]:
all_sample = pd.read_csv('all_sample.csv', sep=",",header=0)

In [ ]:
from sklearn import model_selection, cross_validation
from sklearn.metrics import accuracy_score

In [ ]:
# Let's create X,Y and do a 50/50 split 
X = all_sample.drop(['isphish'], axis=1).set_index('url')
Y = all_sample[['url','isphish']].set_index('url')
# split data into training and test sets
seed = 42
test_size = 0.5
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=test_size, random_state=seed)

## First fit a Random forrest model with imbalanced classes

In [ ]:
# Train the model - 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=5, random_state=0)
model.fit(X_train, np.array(y_train).ravel())

In [ ]:
list(zip(X_test.columns, model.feature_importances_)) #model.feature_importances_

In [ ]:
print(cross_val_score(model, np.array(X_train), np.array(y_train).ravel(), cv=10))

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

dot_data = StringIO()
export_graphviz(model, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [ ]:
print_confusion_matrix(confusion_matrix(y_test, y_pred),list(['Not Phishing','Is Phishing']))

In [ ]:
## Next lets try under sampling 

In [ ]:
rus = RandomUnderSampler(return_indices=True)
X_resampled, y_resampled, idx_resampled = rus.fit_sample(np.array(X), np.array(Y).ravel())

imbal_model = RandomForestClassifier(max_depth=5, random_state=0)
imbal_model.fit(X_resampled, y_resampled)
print(cross_val_score(imbal_model, X_resampled, y_resampled.ravel(), cv=10))

In [ ]:
list(zip(X_test.columns, imbal_model.feature_importances_)) #model.feature_importances_

In [ ]:
from sklearn.metrics import confusion_matrix
#y_pred = imbal_model.predict(X_test)
y_pred = (imbal_model.predict_proba(X_test)[:,1] > 0.9529).astype(bool) 
confusion_matrix(y_test, y_pred)

In [ ]:
print_confusion_matrix(confusion_matrix(y_test, y_pred),list(['Not Phishing','Is Phishing']))

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
ros = RandomOverSampler()
X_rosampled, y_rosampled = ros.fit_sample(np.array(X), np.array(Y).ravel())

imbal_o_model = RandomForestClassifier(max_depth=5, random_state=0)
imbal_o_model.fit(X_rosampled, y_rosampled)
print(cross_val_score(imbal_o_model, X_rosampled, y_rosampled.ravel(), cv=10))

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = imbal_model.predict(X_test)
#y_pred = (imbal_o_model.predict_proba(X_test)[:,1] > 0.95).astype(bool) 
confusion_matrix(y_test, y_pred)
print_confusion_matrix(confusion_matrix(y_test, y_pred),list(['Not Phishing','Is Phishing']))

In [ ]:
list(zip(X_test.columns, imbal_o_model.feature_importances_)) #model.feature_importances_

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = (imbal_o_model.predict_proba(X_test)[:,1] > 0.952).astype(bool) 
confusion_matrix(y_test, y_pred)
print_confusion_matrix(confusion_matrix(y_test, y_pred),list(['Not Phishing','Is Phishing']))